In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# read in beerstyles csv
df = pd.read_csv('allbeerLinksRonessa.csv', encoding = "ISO-8859-1")

In [3]:
df.head()

,abv,beer_name,beer_style,beer_url,brewery,csv_name,match_name,num_ratings,score,style_url
0,5.00,Budweiser,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/29/65/,Anheuser-Busch,americanadjunctlager,americanadjunctlager,6308,2.36,https://www.beeradvocate.com/beer/style/38/
1,4.74,Pabst Blue Ribbon (PBR),American Adjunct Lager,https://www.beeradvocate.com/beer/profile/447/...,Pabst Brewing Company,americanadjunctlager,americanadjunctlager,5683,2.93,https://www.beeradvocate.com/beer/style/38/
2,4.60,Corona Extra,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/75/232/,Grupo Modelo S.A. de C.V.,americanadjunctlager,americanadjunctlager,5072,2.34,https://www.beeradvocate.com/beer/style/38/
3,4.60,Miller High Life,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/105/...,Miller Brewing Co.,americanadjunctlager,americanadjunctlager,3797,2.69,https://www.beeradvocate.com/beer/style/38/
4,4.70,Red Stripe Jamaican Lager,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/221/...,Desnoes & Geddes Limited,americanadjunctlager,americanadjunctlager,3255,2.97,https://www.beeradvocate.com/beer/style/38/


In [4]:
# dfKristine = df.iloc[0:5000]
# dfRonessa = df.iloc[5000:10000]
# dfPaul = df.iloc[10000:15000]
# dfJing = df.iloc[15000:]

In [5]:
# dfJing.to_csv("JingBeer.csv")

In [6]:
# urls = [row['url'] for index,row in df.iterrows()]

In [7]:
# urls

#  beer_dict = {
#                     "beer_name": beer_list[i],
#                     "beer_style": style_names[x],
#                     "beer_url": "https://www.beeradvocate.com" + beer_links[i],
#                     "csv_name": csv_names[x],
#                     "match_name": match_names[x],
#                     "style_url": style_urls[x],
#                     "num_ratings": ratings_list[i],
#                     'score': scores[i],
#                     'abv': abv[i],
#                     'brewery': brewery[i]
#                 }

In [8]:
Name = []
Style = []
Url = []
Score = [] #<span class="ba-ravg">2.36</span>
ScoreClass = []
RatingCount = []
Ranking = []
ReviewCount = []
pDev = [] #percent deviation
BrosScore = []
Brewer = []
Location = []
ABV = []
Availability = []
imageLink = []
csv_name = []
match_name = []
num_ratings = []

beerCount = 1

for index, row in df.iterrows():
    print(row[2])
    name = row['beer_name']
    style = row["beer_style"]
    url = row["beer_url"]
    
    Name.append(name)
    Style.append(style)
    Url.append(url)
    csv_name.append(row['csv_name'])
    match_name.append(row['match_name'])
    num_ratings.append(row['num_ratings'])
    Brewer.append(row['brewery'])
    ABV.append(row['abv'])
    Score.append(row['score'])
    
# test 1 url first
# url = "https://www.beeradvocate.com/beer/profile/29/65/"
# urls = [row['url'] for row in df.iterrows()]

# use this to test 2 urls in a row
# urls = ['https://www.beeradvocate.com/beer/profile/29/65/','https://www.beeradvocate.com/beer/profile/285/39766/']
# for url in urls:
    # go to each beer style page and get the top 50 beers (by count of ratings)
    # response from site
    try:
        print(beerCount)
        response = requests.get(url)

        # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'html.parser')

#         beerScore = soup.find('span', class_='ba-ravg')
#         if (beerScore.text is not None):
#             Score.append(beerScore.text)
#         else:
#             Score.append('')

        scoreclass = soup.find('div', {'id':'score_box'}).find_all('b')
        if (scoreclass[1] is not None):
            ScoreClass.append(scoreclass[1].text)
        else:
            ScoreClass.append('')

#         rating = soup.find('span', class_='ba-ratings')
#         if (rating.text is not None):
#             RatingCount.append(rating.text)
#         else:
#             RatingCount.append('')

        itemstats = soup.find('div', {'id': 'item_stats'}).find_all('dd')
        ranking = itemstats[0].text.replace("#", "").replace(",", "")
        if (ranking is not None):
            Ranking.append(ranking)
        else:
            Ranking.append('')

        reviews = itemstats[1].text.replace(",", "")
        if (reviews is not None):
            ReviewCount.append(reviews)
        else:
            ReviewCount('')

        pdev = re.sub('\s','',itemstats[3].text).replace("%", "")
        if (pdev is not None):
            pDev.append(pdev)
        else:
            pDev.append('')

        brosdiv = soup.find('span', {'class': 'ba-bro_score'})
        #print(brosdiv)
        bros = brosdiv.text.strip()
        if (bros is not None):
            BrosScore.append(bros)
        else:
            BrosScore.append('')

        # get brewer and brewery from infobox    
        infobox = soup.find('div', {'id': 'info_box'}).find_all('a')

#         brewer = infobox[0].find('b').text

#         if (brewer is not None):
#             Brewer.append(brewer)
#         else:
#             Brewer.append('')

        # what needs to be checked for empty here?
        # do i need to empty the list when i'm done using it for next time?
        location = []
        for ahref in infobox:
            if ('/place/' in ahref['href']):
                location.append(ahref.text)

                
        if (len(location) > 1):
            place = ', '.join(location)
        elif (len(location) == 1):
            place = location[0]
        else:
            place = ''
#         print(place)
        Location.append(place)

        # get ABV and availability from infoboxHeadings
        infoboxHeadings = soup.find('div', {'id': 'info_box'}).find_all('b')
#         abv = re.sub('\s','',infoboxHeadings[5].next_sibling).replace("%", "")
#         if (abv is not None):
#             ABV.append(abv)
#         else:
#             ABV.append('')

        availability = re.sub('\s','',infoboxHeadings[6].next_sibling)
        if (availability is not None):
            Availability.append(availability)
        else:
            Availability.append('')

        beerimages = soup.find('div', {'id': 'main_pic_norm'}).find_all('img')
        beerlink = beerimages[0]['src']
        if (beerlink is not None):
            imageLink.append(beerlink)
        else:
            imageLink.append('')

        beerCount += 1
    except:
        print('error!', beerCount, url)
        beerCount += 1 #move to the next one
    

American Adjunct Lager
1
American Adjunct Lager
2
American Adjunct Lager
3
American Adjunct Lager
4
American Adjunct Lager
5
American Adjunct Lager
6
American Adjunct Lager
7
American Adjunct Lager
8
American Adjunct Lager
9
American Adjunct Lager
10
American Adjunct Lager
11
American Adjunct Lager
12
American Adjunct Lager
13
American Adjunct Lager
14
American Adjunct Lager
15
American Adjunct Lager
16
American Adjunct Lager
17
American Adjunct Lager
18
American Adjunct Lager
19
American Adjunct Lager
20
American Adjunct Lager
21
American Adjunct Lager
22
American Adjunct Lager
23
American Adjunct Lager
24
American Adjunct Lager
25
American Adjunct Lager
26
American Adjunct Lager
27
American Adjunct Lager
28
American Adjunct Lager
29
American Adjunct Lager
30
American Adjunct Lager
31
American Adjunct Lager
32
American Adjunct Lager
33
American Adjunct Lager
34
American Adjunct Lager
35
American Adjunct Lager
36
American Adjunct Lager
37
American Adjunct Lager
38
American Adjunct Lage

American Adjunct Lager
309
American Adjunct Lager
310
American Adjunct Lager
311
American Adjunct Lager
312
American Adjunct Lager
313
American Adjunct Lager
314
American Adjunct Lager
315
American Adjunct Lager
316
American Adjunct Lager
317
American Adjunct Lager
318
American Adjunct Lager
319
American Adjunct Lager
320
American Adjunct Lager
321
American Adjunct Lager
322
American Adjunct Lager
323
American Adjunct Lager
324
American Adjunct Lager
325
American Adjunct Lager
326
American Adjunct Lager
327
American Adjunct Lager
328
American Adjunct Lager
329
American Adjunct Lager
330
American Adjunct Lager
331
American Adjunct Lager
332
American Adjunct Lager
333
American Adjunct Lager
334
American Adjunct Lager
335
American Adjunct Lager
336
American Adjunct Lager
337
American Adjunct Lager
338
American Adjunct Lager
339
American Adjunct Lager
340
American Adjunct Lager
341
American Adjunct Lager
342
American Adjunct Lager
343
American Adjunct Lager
344
American Adjunct Lager
345
A

Munich Helles Lager
635
Munich Helles Lager
636
Munich Helles Lager
637
Munich Helles Lager
638
Munich Helles Lager
639
Munich Helles Lager
640
Munich Helles Lager
641
Munich Helles Lager
642
Munich Helles Lager
643
Munich Helles Lager
644
Munich Helles Lager
645
Munich Helles Lager
646
Munich Helles Lager
647
Munich Helles Lager
648
Munich Helles Lager
649
Munich Helles Lager
650
Munich Helles Lager
651
Munich Helles Lager
652
Munich Helles Lager
653
Munich Helles Lager
654
Munich Helles Lager
655
Munich Helles Lager
656
Munich Helles Lager
657
Munich Helles Lager
658
Munich Helles Lager
659
Munich Helles Lager
660
Munich Helles Lager
661
Munich Helles Lager
662
Munich Helles Lager
663
Munich Helles Lager
664
Munich Helles Lager
665
Munich Helles Lager
666
Munich Helles Lager
667
Munich Helles Lager
668
Munich Helles Lager
669
Munich Helles Lager
670
Munich Helles Lager
671
Munich Helles Lager
672
Munich Helles Lager
673
Munich Helles Lager
674
Munich Helles Lager
675
Munich Helles La

German Pilsener
1034
German Pilsener
1035
German Pilsener
1036
German Pilsener
1037
German Pilsener
1038
German Pilsener
1039
German Pilsener
1040
German Pilsener
1041
German Pilsener
1042
German Pilsener
1043
German Pilsener
1044
German Pilsener
1045
German Pilsener
1046
German Pilsener
1047
German Pilsener
1048
German Pilsener
1049
German Pilsener
1050
German Pilsener
1051
German Pilsener
1052
German Pilsener
1053
German Pilsener
1054
German Pilsener
1055
German Pilsener
1056
German Pilsener
1057
German Pilsener
1058
German Pilsener
1059
German Pilsener
1060
German Pilsener
1061
German Pilsener
1062
German Pilsener
1063
German Pilsener
1064
German Pilsener
1065
German Pilsener
1066
German Pilsener
1067
German Pilsener
1068
German Pilsener
1069
German Pilsener
1070
German Pilsener
1071
German Pilsener
1072
German Pilsener
1073
German Pilsener
1074
German Pilsener
1075
German Pilsener
1076
German Pilsener
1077
German Pilsener
1078
German Pilsener
1079
German Pilsener
1080
German Pilsen

Czech Pilsener
1429
Czech Pilsener
1430
Czech Pilsener
1431
Czech Pilsener
1432
Czech Pilsener
1433
Czech Pilsener
1434
Czech Pilsener
1435
Czech Pilsener
1436
Czech Pilsener
1437
Czech Pilsener
1438
Czech Pilsener
1439
Czech Pilsener
1440
Czech Pilsener
1441
Czech Pilsener
1442
Czech Pilsener
1443
Czech Pilsener
1444
Czech Pilsener
1445
Czech Pilsener
1446
Czech Pilsener
1447
Czech Pilsener
1448
Czech Pilsener
1449
Czech Pilsener
1450
Czech Pilsener
1451
Czech Pilsener
1452
Czech Pilsener
1453
Czech Pilsener
1454
Czech Pilsener
1455
Czech Pilsener
1456
Czech Pilsener
1457
Czech Pilsener
1458
Czech Pilsener
1459
Czech Pilsener
1460
Czech Pilsener
1461
Czech Pilsener
1462
Czech Pilsener
1463
Czech Pilsener
1464
Czech Pilsener
1465
Czech Pilsener
1466
Czech Pilsener
1467
Czech Pilsener
1468
Czech Pilsener
1469
Czech Pilsener
1470
Czech Pilsener
1471
Czech Pilsener
1472
Czech Pilsener
1473
Czech Pilsener
1474
Czech Pilsener
1475
Czech Pilsener
1476
Czech Pilsener
1477
Czech Pilsener
1478


...

American Wild Ale
31098
American Wild Ale
31099
American Wild Ale
31100
American Wild Ale
31101
American Wild Ale
31102
American Wild Ale
31103
American Wild Ale
31104
American Wild Ale
31105
American Wild Ale
31106
American Wild Ale
31107
American Wild Ale
31108
American Wild Ale
31109
American Wild Ale
31110
American Wild Ale
31111
American Wild Ale
31112
American Wild Ale
31113
American Wild Ale
31114
American Wild Ale
31115
American Wild Ale
31116
American Wild Ale
31117
American Wild Ale
31118
American Wild Ale
31119
American Wild Ale
31120
American Wild Ale
31121
American Wild Ale
31122
American Wild Ale
31123
American Wild Ale
31124
American Wild Ale
31125
American Wild Ale
31126
American Wild Ale
31127
American Wild Ale
31128
American Wild Ale
31129
American Wild Ale
31130
American Wild Ale
31131
American Wild Ale
31132
American Wild Ale
31133
American Wild Ale
31134
American Wild Ale
31135
American Wild Ale
31136
American Wild Ale
31137
American Wild Ale
31138
American Wild Al

American Wild Ale
31440
American Wild Ale
31441
American Wild Ale
31442
American Wild Ale
31443
American Wild Ale
31444
American Wild Ale
31445
American Wild Ale
31446
American Wild Ale
31447
American Wild Ale
31448
American Wild Ale
31449
American Wild Ale
31450
American Wild Ale
31451
American Wild Ale
31452
American Wild Ale
31453
American Wild Ale
31454
American Wild Ale
31455
American Wild Ale
31456
American Wild Ale
31457
American Wild Ale
31458
American Wild Ale
31459
American Wild Ale
31460
American Wild Ale
31461
American Wild Ale
31462
American Wild Ale
31463
American Wild Ale
31464
American Wild Ale
31465
American Wild Ale
31466
American Wild Ale
31467
American Wild Ale
31468
American Wild Ale
31469
American Wild Ale
31470
American Wild Ale
31471
American Wild Ale
31472
American Wild Ale
31473
American Wild Ale
31474
American Wild Ale
31475
American Wild Ale
31476
American Wild Ale
31477
American Wild Ale
31478
American Wild Ale
31479
American Wild Ale
31480
American Wild Al

Lambic - Fruit
31793
Lambic - Fruit
31794
Lambic - Fruit
31795
Lambic - Fruit
31796
Lambic - Fruit
31797
Lambic - Fruit
31798
Lambic - Fruit
31799
Lambic - Fruit
31800
Lambic - Fruit
31801
Lambic - Fruit
31802
Lambic - Fruit
31803
Lambic - Fruit
31804
Lambic - Fruit
31805
Lambic - Fruit
31806
Lambic - Fruit
31807
Lambic - Fruit
31808
Lambic - Fruit
31809
Lambic - Fruit
31810
Lambic - Fruit
31811
Lambic - Fruit
31812
Lambic - Fruit
31813
Lambic - Fruit
31814
Lambic - Fruit
31815
Lambic - Fruit
31816
Lambic - Fruit
31817
Lambic - Fruit
31818
Lambic - Fruit
31819
Lambic - Fruit
31820
Lambic - Fruit
31821
Lambic - Fruit
31822
Lambic - Fruit
31823
Lambic - Fruit
31824
Lambic - Fruit
31825
Lambic - Fruit
31826
Lambic - Fruit
31827
Lambic - Fruit
31828
Lambic - Fruit
31829
Lambic - Fruit
31830
Lambic - Fruit
31831
Lambic - Fruit
31832
Lambic - Fruit
31833
Lambic - Fruit
31834
Lambic - Fruit
31835
Lambic - Fruit
31836
Lambic - Fruit
31837
Lambic - Fruit
31838
Lambic - Fruit
31839
Lambic - Frui

Pumpkin Ale
32225
Pumpkin Ale
32226
Pumpkin Ale
32227
Pumpkin Ale
32228
Pumpkin Ale
32229
Pumpkin Ale
32230
Pumpkin Ale
32231
Pumpkin Ale
32232
Pumpkin Ale
32233
Pumpkin Ale
32234
Pumpkin Ale
32235
Pumpkin Ale
32236
Pumpkin Ale
32237
Pumpkin Ale
32238
Pumpkin Ale
32239
Pumpkin Ale
32240
Pumpkin Ale
32241
Pumpkin Ale
32242
Pumpkin Ale
32243
Pumpkin Ale
32244
Pumpkin Ale
32245
Pumpkin Ale
32246
Pumpkin Ale
32247
Pumpkin Ale
32248
Pumpkin Ale
32249
Pumpkin Ale
32250
Pumpkin Ale
32251
Pumpkin Ale
32252
Pumpkin Ale
32253
Pumpkin Ale
32254
Pumpkin Ale
32255
Pumpkin Ale
32256
Pumpkin Ale
32257
Pumpkin Ale
32258
Pumpkin Ale
32259
Pumpkin Ale
32260
Pumpkin Ale
32261
Pumpkin Ale
32262
Pumpkin Ale
32263
Pumpkin Ale
32264
Pumpkin Ale
32265
Pumpkin Ale
32266
Pumpkin Ale
32267
Pumpkin Ale
32268
Pumpkin Ale
32269
Pumpkin Ale
32270
Pumpkin Ale
32271
Pumpkin Ale
32272
Pumpkin Ale
32273
Pumpkin Ale
32274
Pumpkin Ale
32275
Pumpkin Ale
32276
Pumpkin Ale
32277
Pumpkin Ale
32278
Pumpkin Ale
32279
Pumpkin Al

Smoked Beer
32726
Smoked Beer
32727
Smoked Beer
32728
Smoked Beer
32729
Smoked Beer
32730
Smoked Beer
32731
Smoked Beer
32732
Smoked Beer
32733
Smoked Beer
32734
Smoked Beer
32735
Smoked Beer
32736
Smoked Beer
32737
Smoked Beer
32738
Smoked Beer
32739
Smoked Beer
32740
Smoked Beer
32741
Smoked Beer
32742
Smoked Beer
32743
Smoked Beer
32744
Smoked Beer
32745
Smoked Beer
32746
Smoked Beer
32747
Smoked Beer
32748
Smoked Beer
32749
Smoked Beer
32750
Smoked Beer
32751
Smoked Beer
32752
Smoked Beer
32753
Smoked Beer
32754
Smoked Beer
32755
Smoked Beer
32756
Smoked Beer
32757
Smoked Beer
32758
Smoked Beer
32759
Smoked Beer
32760
Smoked Beer
32761
Smoked Beer
32762
Smoked Beer
32763
Smoked Beer
32764
Smoked Beer
32765
Smoked Beer
32766
Smoked Beer
32767
Smoked Beer
32768
Smoked Beer
32769
Smoked Beer
32770
Smoked Beer
32771
Smoked Beer
32772
Smoked Beer
32773
Smoked Beer
32774
Smoked Beer
32775
Smoked Beer
32776
Smoked Beer
32777
Smoked Beer
32778
Smoked Beer
32779
Smoked Beer
32780
Smoked Bee

In [9]:
# scoreclass = soup.find('div', {'id':'score_box'}).find_all('b')
# sc = scoreclass[1].text
# print (sc)
# print(imageLink)

In [10]:
num_ratings

[6308,
 5683,
 5072,
 3797,
 3255,
 2967,
 2924,
 2766,
 2494,
 2316,
 2098,
 2069,
 2064,
 1993,
 1888,
 1794,
 1760,
 1725,
 1433,
 1428,
 1407,
 1301,
 1257,
 1148,
 1060,
 883,
 821,
 804,
 780,
 775,
 764,
 728,
 716,
 680,
 612,
 592,
 588,
 572,
 553,
 552,
 543,
 528,
 513,
 497,
 484,
 480,
 474,
 443,
 416,
 384,
 373,
 337,
 328,
 317,
 317,
 317,
 302,
 286,
 280,
 271,
 270,
 266,
 257,
 251,
 246,
 240,
 239,
 227,
 216,
 214,
 208,
 204,
 201,
 197,
 195,
 188,
 188,
 182,
 179,
 174,
 172,
 169,
 168,
 167,
 165,
 161,
 159,
 156,
 154,
 152,
 148,
 147,
 141,
 140,
 133,
 132,
 131,
 130,
 129,
 128,
 127,
 127,
 120,
 113,
 113,
 113,
 112,
 109,
 107,
 103,
 103,
 102,
 100,
 99,
 99,
 99,
 96,
 95,
 95,
 89,
 88,
 88,
 88,
 87,
 87,
 85,
 84,
 84,
 84,
 84,
 83,
 83,
 83,
 81,
 80,
 80,
 78,
 78,
 77,
 76,
 76,
 75,
 75,
 75,
 75,
 74,
 73,
 73,
 72,
 71,
 70,
 70,
 70,
 69,
 68,
 68,
 66,
 65,
 62,
 60,
 59,
 59,
 59,
 59,
 58,
 58,
 58,
 58,
 58,
 57,
 57,
 57,
 5

In [21]:
len(num_ratings)

32781

In [61]:
num_ratings[6139]

106

In [ ]:
#delete 6139 from num_ratings, Name, Url, Style, Score, ABV, brewer, csv_name, match_name

In [40]:
len(Name)

32781

In [99]:
# del num_ratings[6139]
# del Name[6139]
# del Url[6139]
# del Style[6139]
# del Score[6139]
# del ABV[6139]
# del Brewer[6139]
# del csv_name[6139]
# del match_name[6139]

In [58]:
Name[6139]

'Local Harvest Ale'

In [60]:
Url[6140]

'https://www.beeradvocate.com/beer/profile/494/20910/'

In [59]:
Ranking[6139]

'41842'

In [72]:
len(Url)

32780

In [73]:
len(Style)

32780

In [74]:
len(Score)

32780

In [75]:
len(ABV)

32780

In [77]:
len(Brewer)

32780

In [78]:
len(csv_name)

32780

In [79]:
len(match_name)

32780

In [80]:
len(pDev)

32780

In [81]:
len(BrosScore)

32780

In [82]:
len(ScoreClass)

32780

In [83]:
len(Ranking)

32780

In [84]:
len(ReviewCount)

32780

In [85]:
len(pDev)

32780

In [86]:
len(Availability)

32780

In [87]:
len(imageLink)

32781

In [90]:
# del imageLink[32780]

In [52]:
# testList = imageLink

In [55]:
# testList.append("none")

In [57]:
# testingdf = pd.DataFrame({"Image Link": testList, "URL": Url})
# testingdf.to_csv("id_missing.csv")

'Year-round'

In [31]:
ABV[32780]

'6.40'

In [37]:
Availability[32779]

'Fall'

In [91]:
featuredf = pd.DataFrame({"Name": Name, "URL": Url, "Style": Style, "Score":Score,"ScoreClass":ScoreClass,
                          "Ranking": Ranking,"ReviewCount": ReviewCount,"pDev": pDev,
                         "BrosScore": BrosScore,"Brewer": Brewer,"Location": Location,"ABV": ABV,
                         "Availaility": Availability,"ImageLink": imageLink, "csv_name": csv_name, "match_name": match_name, "num_ratings": num_ratings})

In [92]:
featuredf

,ABV,Availaility,Brewer,BrosScore,ImageLink,Location,Name,Ranking,ReviewCount,Score,ScoreClass,Style,URL,csv_name,match_name,num_ratings,pDev
0,5.00,Year-round,Anheuser-Busch,0,https://cdn.beeradvocate.com/im/beers/65.jpg,"Missouri, United States",Budweiser,45345,1713,2.36,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/29/65/,americanadjunctlager,americanadjunctlager,6308,33.47
1,4.74,Year-round,Pabst Brewing Company,3.65,https://cdn.beeradvocate.com/im/beers/1331.jpg,"California, United States",Pabst Blue Ribbon (PBR),45085,1753,2.93,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/447/...,americanadjunctlager,americanadjunctlager,5683,25.26
2,4.60,Year-round,Grupo Modelo S.A. de C.V.,1.89,https://cdn.beeradvocate.com/im/beers/232.jpg,Mexico,Corona Extra,45347,1261,2.34,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/75/232/,americanadjunctlager,americanadjunctlager,5072,31.62
3,4.60,Year-round,Miller Brewing Co.,3.53,https://cdn.beeradvocate.com/im/beers/580.jpg,"Wisconsin, United States",Miller High Life,45258,1158,2.69,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/105/...,americanadjunctlager,americanadjunctlager,3797,28.62
4,4.70,Year-round,Desnoes & Geddes Limited,3.39,https://cdn.beeradvocate.com/im/beers/689.jpg,Jamaica,Red Stripe Jamaican Lager,45039,985,2.97,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/221/...,americanadjunctlager,americanadjunctlager,3255,20.88
5,5.00,Year-round,Coors Brewing Company (Molson-Coors),3.05,https://cdn.beeradvocate.com/im/beers/1276.jpg,"Colorado, United States",Coors Banquet,45298,741,2.59,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/306/...,americanadjunctlager,americanadjunctlager,2967,28.57
6,4.60,Year-round,Latrobe Brewing Co.,2.86,https://cdn.beeradvocate.com/im/beers/567.jpg,"Missouri, United States",Rolling Rock Extra Pale,45297,853,2.59,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/174/...,americanadjunctlager,americanadjunctlager,2924,27.03
7,4.30,Year-round,"CervecerÃ­a CuauhtÃ©moc Moctezuma, S.A. de C.V.",2.65,https://cdn.beeradvocate.com/im/beers/1907.jpg,Mexico,Dos Equis Special Lager,45226,620,2.74,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/57/1...,americanadjunctlager,americanadjunctlager,2766,22.99
8,4.66,Year-round,Miller Brewing Co.,2.19,https://cdn.beeradvocate.com/im/beers/2280.jpg,"Wisconsin, United States",Miller Genuine Draft,45352,662,2.27,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/105/...,americanadjunctlager,americanadjunctlager,2494,30.84
9,4.40,Year-round,Grupo Modelo S.A. de C.V.,3.4,https://cdn.beeradvocate.com/im/beers/1321.jpg,Mexico,Modelo Especial,45172,601,2.82,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/75/1...,americanadjunctlager,americanadjunctlager,2316,22.34


In [93]:
# featuredf['Ranking'] = featuredf['Ranking'].str.replace(",", "")
# featuredf['RatingCount'] = featuredf['RatingCount'].str.replace(",", "")
# featuredf['ReviewCount'] = featuredf['ReviewCount'].str.replace(",", "")

In [94]:
# save to csv
featuredf.to_csv('AllScrapedBeerFeatures2.csv', index=False)

In [95]:
# read in beerstyles csv
test = pd.read_csv('AllBeersEachStyle.csv', encoding = "ISO-8859-1")

In [96]:
test['urlHack'] = test['url'] + ' '

In [97]:
test.to_csv('AllBeersEachStyleHack2.csv')

In [98]:
featuredf['beerid'] = featuredf['ImageLink'].replace(['https://cdn.beeradvocate.com/im/beers/',',jpg'],'', inplace=True)
featuredf

,ABV,Availaility,Brewer,BrosScore,ImageLink,Location,Name,Ranking,ReviewCount,Score,ScoreClass,Style,URL,csv_name,match_name,num_ratings,pDev,beerid
0,5.00,Year-round,Anheuser-Busch,0,https://cdn.beeradvocate.com/im/beers/65.jpg,"Missouri, United States",Budweiser,45345,1713,2.36,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/29/65/,americanadjunctlager,americanadjunctlager,6308,33.47,None
1,4.74,Year-round,Pabst Brewing Company,3.65,https://cdn.beeradvocate.com/im/beers/1331.jpg,"California, United States",Pabst Blue Ribbon (PBR),45085,1753,2.93,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/447/...,americanadjunctlager,americanadjunctlager,5683,25.26,None
2,4.60,Year-round,Grupo Modelo S.A. de C.V.,1.89,https://cdn.beeradvocate.com/im/beers/232.jpg,Mexico,Corona Extra,45347,1261,2.34,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/75/232/,americanadjunctlager,americanadjunctlager,5072,31.62,None
3,4.60,Year-round,Miller Brewing Co.,3.53,https://cdn.beeradvocate.com/im/beers/580.jpg,"Wisconsin, United States",Miller High Life,45258,1158,2.69,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/105/...,americanadjunctlager,americanadjunctlager,3797,28.62,None
4,4.70,Year-round,Desnoes & Geddes Limited,3.39,https://cdn.beeradvocate.com/im/beers/689.jpg,Jamaica,Red Stripe Jamaican Lager,45039,985,2.97,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/221/...,americanadjunctlager,americanadjunctlager,3255,20.88,None
5,5.00,Year-round,Coors Brewing Company (Molson-Coors),3.05,https://cdn.beeradvocate.com/im/beers/1276.jpg,"Colorado, United States",Coors Banquet,45298,741,2.59,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/306/...,americanadjunctlager,americanadjunctlager,2967,28.57,None
6,4.60,Year-round,Latrobe Brewing Co.,2.86,https://cdn.beeradvocate.com/im/beers/567.jpg,"Missouri, United States",Rolling Rock Extra Pale,45297,853,2.59,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/174/...,americanadjunctlager,americanadjunctlager,2924,27.03,None
7,4.30,Year-round,"CervecerÃ­a CuauhtÃ©moc Moctezuma, S.A. de C.V.",2.65,https://cdn.beeradvocate.com/im/beers/1907.jpg,Mexico,Dos Equis Special Lager,45226,620,2.74,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/57/1...,americanadjunctlager,americanadjunctlager,2766,22.99,None
8,4.66,Year-round,Miller Brewing Co.,2.19,https://cdn.beeradvocate.com/im/beers/2280.jpg,"Wisconsin, United States",Miller Genuine Draft,45352,662,2.27,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/105/...,americanadjunctlager,americanadjunctlager,2494,30.84,None
9,4.40,Year-round,Grupo Modelo S.A. de C.V.,3.4,https://cdn.beeradvocate.com/im/beers/1321.jpg,Mexico,Modelo Especial,45172,601,2.82,Poor,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/75/1...,americanadjunctlager,americanadjunctlager,2316,22.34,None
